In [1]:
import gym
import numpy as np
import time
from IPython import display

In [2]:
env = gym.make('FrozenLake8x8-v0')

In [3]:
env.P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 8, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 8, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 8, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 9, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 9, 0.0, False),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 9, 0.0, False),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},


In [4]:
env.observation_space.n

64

In [5]:
env.action_space.n

4

In [6]:
def value_iteration(env, max_iters, gamma):
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Compute the value for state
        for state in range(env.observation_space.n):
            q_values = []
            # Compute the q-value for each action
            for action in range(env.action_space.n):
                q_value = 0
                # Loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                
                q_values.append(q_value)
            
            # Select the best action
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [7]:
v_values = value_iteration(env, max_iters=1000, gamma=0.9)

Converged at 117-th iteration.


In [8]:
v_values

array([0.00641104, 0.00854808, 0.01230044, 0.01778942, 0.02508214,
       0.03247089, 0.03957134, 0.04297844, 0.00602405, 0.00764512,
       0.01091162, 0.01642654, 0.02605411, 0.03619409, 0.0493547 ,
       0.05730461, 0.00509024, 0.0058532 , 0.00677534, 0.        ,
       0.02557084, 0.03882139, 0.06763973, 0.08435607, 0.0042256 ,
       0.00476954, 0.00581968, 0.0078541 , 0.02036065, 0.        ,
       0.09175501, 0.12919111, 0.00318093, 0.00319659, 0.00270488,
       0.        , 0.0344439 , 0.06195145, 0.10901921, 0.20969093,
       0.00186915, 0.        , 0.        , 0.01085079, 0.03250092,
       0.06304172, 0.        , 0.36008773, 0.00118046, 0.        ,
       0.00137717, 0.00366839, 0.        , 0.11568671, 0.        ,
       0.63051379, 0.00088531, 0.00077462, 0.00092218, 0.        ,
       0.13824885, 0.32258065, 0.61443932, 0.        ])

In [9]:
def policy_extraction(env, v_values, gamma=0.9):
    policy = np.zeros(env.observation_space.n, dtype=np.int)

    # Compute the best action for each state in the game
    # Compute q-value for each (state-action) pair in the game
    for state in range(env.observation_space.n):
        q_values = []
        # Compute q_value for each action
        for action in range(env.action_space.n):
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            q_values.append(q_value)
        
        # Select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
    
    return policy

In [10]:
policy = policy_extraction(env, v_values, gamma=0.9)

In [11]:
policy

array([3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 1, 3, 3, 0, 0, 2, 3,
       2, 1, 3, 3, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 2, 1, 3, 2, 0, 0, 0, 1,
       3, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 1, 1, 0])

In [12]:
def play(env, policy):
    state = env.reset()
    total_reward = 0
    done = False
    steps = 0
    #time.sleep(1)
    #display.clear_output(wait=True)
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        #print(f'Step {steps}')
        #env.render()
        #time.sleep(0.2)
        #if not done:
        #    display.clear_output(wait=True)
        state = next_state

    return total_reward

In [14]:
def play_multiple_times(env, policy, max_episodes):
    success = 0

    for i in range(max_episodes):
        reward = play(env, policy)

        if reward > 0:
            success += 1
    
    print(f'Number of successes: {success}/{max_episodes}')

In [17]:
play_multiple_times(env, policy, 1000)

Number of successes: 756/1000
